In [35]:
import time
import datetime
from collections import namedtuple

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [36]:
# display full, non-truncated rows
pd.set_option('display.max_colwidth', -1)

brasil_data = pd.read_csv('altered_deter_public_amz_deter_public.csv', header=0)

d:\csfiles\lambda\git\epicentral\venv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [37]:
brasil_data

,CLASSNAME,QUADRANT,PATH_ROW,VIEW_DATE,SENSOR,SATELLITE,AREAUCKM,UC,AREAMUNKM,MUNICIPALI,...,event1_date_end,event1_km2,event1_lat,event1_lon,event2_yes_within_2weeks,event2_date_start,event2_date_end,event2_km2,event2_lat,event2_lon
0,CICATRIZ_DE_QUEIMADA,NaN,165105,7/27/2020,AWFI,CBERS-4,0.000000,NaN,0.075231,Novo Repartimento,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CICATRIZ_DE_QUEIMADA,NaN,168105,1/17/2018,AWFI,CBERS-4,0.000000,NaN,0.344234,Altamira,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CICATRIZ_DE_QUEIMADA,NaN,168105,1/17/2018,AWFI,CBERS-4,0.000000,NaN,4.788569,Brasil Novo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CICATRIZ_DE_QUEIMADA,NaN,168105,1/17/2018,AWFI,CBERS-4,0.000000,NaN,1.067603,Altamira,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CICATRIZ_DE_QUEIMADA,NaN,168105,1/17/2018,AWFI,CBERS-4,0.000000,NaN,0.067680,Alenquer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186408,MINERACAO,NaN,210124,6/24/2020,WFI,CBERS-4A,0.000000,NaN,0.110526,Cumaru do Norte,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186409,MINERACAO,NaN,165105,7/27/2020,AWFI,CBERS-4,0.000000,NaN,0.095688,Bannach,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186410,MINERACAO,NaN,165105,7/27/2020,AWFI,CBERS-4,0.000000,NaN,0.063135,Bannach,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186411,MINERACAO,NaN,169105,1/14/2018,AWFI,CBERS-4,0.121459,├üREA DE PROTE├ç├âO AMBIENTAL DO TAPAJ├ôS,0.121459,Jacareacanga,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
brasil_data.columns

Index(['CLASSNAME', 'QUADRANT', 'PATH_ROW', 'VIEW_DATE', 'SENSOR', 'SATELLITE',
       'AREAUCKM', 'UC', 'AREAMUNKM', 'MUNICIPALI', 'UF', 'LAT', 'LONG',
       'base_url', 'name', 'event1_date_start', 'event1_date_end',
       'event1_km2', 'event1_lat', 'event1_lon', 'event2_yes_within_2weeks',
       'event2_date_start', 'event2_date_end', 'event2_km2', 'event2_lat',
       'event2_lon'],
      dtype='object')

In [39]:
# Convert VIEW DATE to datetimes
brasil_data['VIEW_DATE'] = brasil_data['VIEW_DATE'].apply(lambda row: datetime.datetime.strptime(row, '%m/%d/%Y'))

In [40]:
def create_times(date):
    '''
    Takes in a date as a string, and returns a tuple of (two_weeks_before, date) in Unix time
    '''
    view_date = time.mktime(date.timetuple()) * 1000
    before_date = view_date -(604800 * 2 * 1000)
    return (int(before_date), int(view_date))

In [41]:
# inserting the UNIX_TIMES (2WeeksPrior, ViewDate) into the dataframe after the VIEW_DATE column
brasil_data.insert(4, 'UNIX_TIMES', brasil_data.apply(lambda row: create_times(row['VIEW_DATE']), axis=1))

In [44]:
brasil_data.columns

Index(['CLASSNAME', 'QUADRANT', 'PATH_ROW', 'VIEW_DATE', 'UNIX_TIMES',
       'SENSOR', 'SATELLITE', 'AREAUCKM', 'UC', 'AREAMUNKM', 'MUNICIPALI',
       'UF', 'LAT', 'LONG', 'base_url', 'name', 'event1_date_start',
       'event1_date_end', 'event1_km2', 'event1_lat', 'event1_lon',
       'event2_yes_within_2weeks', 'event2_date_start', 'event2_date_end',
       'event2_km2', 'event2_lat', 'event2_lon'],
      dtype='object')

In [45]:
##### START GILLES GEO CODE ####
geometry = [Point(xy) for xy in zip(brasil_data.LAT, brasil_data.LONG)]

In [46]:
brasil_data_gdf = gpd.GeoDataFrame(brasil_data, geometry=geometry)

In [47]:
brasil_data_gdf.crs = 'epsg:4326'

In [48]:
deg_to_meters_lat_minus_seven = 110590
five_km_in_deg = 5000/110590.
def create_buffer(row):
    dist = five_km_in_deg
    point = row["geometry"]
    row["geometry"] = point.buffer(dist)
    return row

In [49]:
brasil_data_gdf.columns

Index(['CLASSNAME', 'QUADRANT', 'PATH_ROW', 'VIEW_DATE', 'UNIX_TIMES',
       'SENSOR', 'SATELLITE', 'AREAUCKM', 'UC', 'AREAMUNKM', 'MUNICIPALI',
       'UF', 'LAT', 'LONG', 'base_url', 'name', 'event1_date_start',
       'event1_date_end', 'event1_km2', 'event1_lat', 'event1_lon',
       'event2_yes_within_2weeks', 'event2_date_start', 'event2_date_end',
       'event2_km2', 'event2_lat', 'event2_lon', 'geometry'],
      dtype='object')

In [50]:
brasil_data_buffer_gdf = brasil_data_gdf.apply(create_buffer,axis=1)

In [51]:
brasil_data_buffer_gdf.crs = 'epsg:4326'

In [52]:
#brasil_data_buffer_gdf.to_file('test_4326.shp')

In [53]:
brasil_data_buffer_gdf['wkt'] = brasil_data_buffer_gdf.apply(lambda row: row.geometry.simplify(0.0005).wkt.replace(' ',''), axis=1)

In [54]:
brasil_data_buffer_gdf.columns

Index(['CLASSNAME', 'QUADRANT', 'PATH_ROW', 'VIEW_DATE', 'UNIX_TIMES',
       'SENSOR', 'SATELLITE', 'AREAUCKM', 'UC', 'AREAMUNKM', 'MUNICIPALI',
       'UF', 'LAT', 'LONG', 'base_url', 'name', 'event1_date_start',
       'event1_date_end', 'event1_km2', 'event1_lat', 'event1_lon',
       'event2_yes_within_2weeks', 'event2_date_start', 'event2_date_end',
       'event2_km2', 'event2_lat', 'event2_lon', 'geometry', 'wkt'],
      dtype='object')

In [55]:
### https://www.planet.com/explorer/#/mode/compare/interval/1%20day/center/-55.348,-7.247/zoom/15.95/geometry/POLYGON((-55.34521+-7.250297,-55.332357+-7.254115,-55.333663+-7.240683,-55.353252+-7.240887,-55.34521+-7.250297))/items/PSScene4Band%3A20201015_143124_84_1069/comparing/result::PSScene4Band:1601164800000,result::PSScene4Band:1602720000000

In [56]:
### URL NEEDS FIXING ###
brasil_data_buffer_gdf['base_url'] = brasil_data_buffer_gdf.apply(lambda row:
f"https://www.planet.com/explorer/#/mode/compare/interval/1%20day/center/{row['LAT']:.3f},{row['LONG']:.3f}/zoom/9/geometry/{row['wkt']}/items/PSScene4Band%3A20201015_143124_84_1069/comparing/result::PSScene4Band:{row['UNIX_TIMES'][0]},result::PSScene4Band:{row['UNIX_TIMES'][1]}", axis=1)


In [57]:
brasil_data_buffer_gdf['base_url']

0         https://www.planet.com/explorer/#/mode/compare/interval/1%20day/center/-50.471,-4.918/zoom/9/geometry/POLYGON((-50.42598795551135-4.91754,-50.4268566922686-4.926360432318298,-50.42942951747395-4.93484190036916,-50.43360755889762-4.942658466091853,-50.43923025675077-4.949509743249234,-50.44608153390815-4.955132441102385,-50.45389809963084-4.959310482526055,-50.4623795676817-4.961883307731405,-50.4712-4.962752044488652,-50.4800204323183-4.961883307731405,-50.48850190036917-4.959310482526055,-50.49631846609186-4.955132441102385,-50.50316974324924-4.949509743249234,-50.50879244110239-4.942658466091853,-50.51297048252606-4.93484190036916,-50.51554330773141-4.926360432318298,-50.51641204448865-4.91754,-50.51554330773141-4.908719567681701,-50.51297048252606-4.900238099630839,-50.50879244110239-4.892421533908147,-50.50316974324924-4.885570256750766,-50.49631846609186-4.879947558897615,-50.48850190036917-4.875769517473945,-50.48002043231831-4.873196692268595,-50.4712-4.872327955511348

In [58]:
brasil_data_buffer_gdf.head()

,CLASSNAME,QUADRANT,PATH_ROW,VIEW_DATE,UNIX_TIMES,SENSOR,SATELLITE,AREAUCKM,UC,AREAMUNKM,...,event1_lat,event1_lon,event2_yes_within_2weeks,event2_date_start,event2_date_end,event2_km2,event2_lat,event2_lon,geometry,wkt
0,CICATRIZ_DE_QUEIMADA,NaN,165105,2020-07-27,"(1594623600000, 1595833200000)",AWFI,CBERS-4,0.0,NaN,0.075231,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-50.42599 -4.91754, -50.42621 -4.92197, -50.42686 -4.92636, -50.42793 -4.93066, -50.42943 -4.93484, -50.43133 -4.93885, -50.43361 -4.94266, -50.43625 -4.94622, -50.43923 -4.94951, -50.44252 -4.95249, -50.44608 -4.95513, -50.44989 -4.95741, -50.45390 -4.95931, -50.45808 -4.96081, -50.46238 -4.96188, -50.46677 -4.96253, -50.47120 -4.96275, -50.47563 -4.96253, -50.48002 -4.96188, -50.48432 -4.96081, -50.48850 -4.95931, -50.49251 -4.95741, -50.49632 -4.95513, -50.49988 -4.95249, -50.50317 -4.94951, -50.50615 -4.94622, -50.50879 -4.94266, -50.51107 -4.93885, -50.51297 -4.93484, -50.51447 -4.93066, -50.51554 -4.92636, -50.51619 -4.92197, -50.51641 -4.91754, -50.51619 -4.91311, -50.51554 -4.90872, -50.51447 -4.90442, -50.51297 -4.90024, -50.51107 -4.89623, -50.50879 -4.89242, -50.50615 -4.88886, -50.50317 -4.88557, -50.49988 -4.88259, -50.49632 -4.87995, -50.49251 -4.87767, -50.48850 -4.87577, -50.48432 -4.87427, -50.48002 -4.87320, -50.47563 -4.87255, -50.47120 -4.87233, -50.46677 -4.87255, -50.46238 -4.87320, -50.45808 -4.87427, -50.45390 -4.87577, -50.44989 -4.87767, -50.44608 -4.87995, -50.44252 -4.88259, -50.43923 -4.88557, -50.43625 -4.88886, -50.43361 -4.89242, -50.43133 -4.89623, -50.42943 -4.90024, -50.42793 -4.90442, -50.42686 -4.90872, -50.42621 -4.91311, -50.42599 -4.91754))","POLYGON((-50.42598795551135-4.91754,-50.4268566922686-4.926360432318298,-50.42942951747395-4.93484190036916,-50.43360755889762-4.942658466091853,-50.43923025675077-4.949509743249234,-50.44608153390815-4.955132441102385,-50.45389809963084-4.959310482526055,-50.4623795676817-4.961883307731405,-50.4712-4.962752044488652,-50.4800204323183-4.961883307731405,-50.48850190036917-4.959310482526055,-50.49631846609186-4.955132441102385,-50.50316974324924-4.949509743249234,-50.50879244110239-4.942658466091853,-50.51297048252606-4.93484190036916,-50.51554330773141-4.926360432318298,-50.51641204448865-4.91754,-50.51554330773141-4.908719567681701,-50.51297048252606-4.900238099630839,-50.50879244110239-4.892421533908147,-50.50316974324924-4.885570256750766,-50.49631846609186-4.879947558897615,-50.48850190036917-4.875769517473945,-50.48002043231831-4.873196692268595,-50.4712-4.872327955511348,-50.46237956768171-4.873196692268595,-50.45389809963084-4.875769517473945,-50.44608153390815-4.879947558897615,-50.43923025675077-4.885570256750766,-50.43360755889762-4.892421533908147,-50.42942951747395-4.900238099630839,-50.4268566922686-4.908719567681701,-50.42598795551135-4.91754))"
1,CICATRIZ_DE_QUEIMADA,NaN,168105,2018-01-17,"(1514966400000, 1516176000000)",AWFI,CBERS-4,0.0,NaN,0.344234,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-55.01139 -6.91208, -55.01161 -6.91651, -55.01226 -6.92090, -55.01333 -6.92520, -55.01483 -6.92938, -55.01673 -6.93339, -55.01901 -6.93720, -55.02165 -6.94076, -55.02463 -6.94405, -55.02792 -6.94703, -55.03148 -6.94967, -55.03529 -6.95195, -55.03930 -6.95385, -55.04348 -6.95535, -55.04778 -6.95642, -55.05217 -6.95707, -55.05660 -6.95729, -55.06103 -6.95707, -55.06542 -6.95642, -55.06972 -6.95535, -55.07390 -6.95385, -55.07791 -6.95195, -55.08172 -6.94967, -55.08528 -6.94703, -55.08857 -6.94405, -55.09155 -6.94076, -55.09419 -6.93720, -55.09647 -6.93339, -55.09837 -6.92938, -55.09987 -6.92520, -55.10094 -6.92090, -55.10159 -6.91651, -55.10181 -6.91208, -55.10159 -6.90765, -55.10094 -6.90326, -55.09987 -6.89896, -55.09837 -6.89478, -55.09647 -6.89077, -55.09419 -6.88696, -55.09155 -6.88340, -55.08857 -6.88011, -55.08528 -6.87713, -55.08172 -6.87449, -55.07791 -6.87221, -55.07390 -6.87031, -55.06972 -6.86881, -55.06542 -6.86774, -55.06103 -6.86709, -55.05660 -6.86687, -55.05217 

In [59]:
seven_month_brasil_data = brasil_data_buffer_gdf[brasil_data_buffer_gdf['VIEW_DATE'] > '2020-03-15']

In [60]:
seven_month_brasil_data

,CLASSNAME,QUADRANT,PATH_ROW,VIEW_DATE,UNIX_TIMES,SENSOR,SATELLITE,AREAUCKM,UC,AREAMUNKM,...,event1_lat,event1_lon,event2_yes_within_2weeks,event2_date_start,event2_date_end,event2_km2,event2_lat,event2_lon,geometry,wkt
0,CICATRIZ_DE_QUEIMADA,NaN,165105,2020-07-27,"(1594623600000, 1595833200000)",AWFI,CBERS-4,0.0,NaN,0.075231,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-50.42599 -4.91754, -50.42621 -4.92197, -50.42686 -4.92636, -50.42793 -4.93066, -50.42943 -4.93484, -50.43133 -4.93885, -50.43361 -4.94266, -50.43625 -4.94622, -50.43923 -4.94951, -50.44252 -4.95249, -50.44608 -4.95513, -50.44989 -4.95741, -50.45390 -4.95931, -50.45808 -4.96081, -50.46238 -4.96188, -50.46677 -4.96253, -50.47120 -4.96275, -50.47563 -4.96253, -50.48002 -4.96188, -50.48432 -4.96081, -50.48850 -4.95931, -50.49251 -4.95741, -50.49632 -4.95513, -50.49988 -4.95249, -50.50317 -4.94951, -50.50615 -4.94622, -50.50879 -4.94266, -50.51107 -4.93885, -50.51297 -4.93484, -50.51447 -4.93066, -50.51554 -4.92636, -50.51619 -4.92197, -50.51641 -4.91754, -50.51619 -4.91311, -50.51554 -4.90872, -50.51447 -4.90442, -50.51297 -4.90024, -50.51107 -4.89623, -50.50879 -4.89242, -50.50615 -4.88886, -50.50317 -4.88557, -50.49988 -4.88259, -50.49632 -4.87995, -50.49251 -4.87767, -50.48850 -4.87577, -50.48432 -4.87427, -50.48002 -4.87320, -50.47563 -4.87255, -50.47120 -4.87233, -50.46677 -4.87255, -50.46238 -4.87320, -50.45808 -4.87427, -50.45390 -4.87577, -50.44989 -4.87767, -50.44608 -4.87995, -50.44252 -4.88259, -50.43923 -4.88557, -50.43625 -4.88886, -50.43361 -4.89242, -50.43133 -4.89623, -50.42943 -4.90024, -50.42793 -4.90442, -50.42686 -4.90872, -50.42621 -4.91311, -50.42599 -4.91754))","POLYGON((-50.42598795551135-4.91754,-50.4268566922686-4.926360432318298,-50.42942951747395-4.93484190036916,-50.43360755889762-4.942658466091853,-50.43923025675077-4.949509743249234,-50.44608153390815-4.955132441102385,-50.45389809963084-4.959310482526055,-50.4623795676817-4.961883307731405,-50.4712-4.962752044488652,-50.4800204323183-4.961883307731405,-50.48850190036917-4.959310482526055,-50.49631846609186-4.955132441102385,-50.50316974324924-4.949509743249234,-50.50879244110239-4.942658466091853,-50.51297048252606-4.93484190036916,-50.51554330773141-4.926360432318298,-50.51641204448865-4.91754,-50.51554330773141-4.908719567681701,-50.51297048252606-4.900238099630839,-50.50879244110239-4.892421533908147,-50.50316974324924-4.885570256750766,-50.49631846609186-4.879947558897615,-50.48850190036917-4.875769517473945,-50.48002043231831-4.873196692268595,-50.4712-4.872327955511348,-50.46237956768171-4.873196692268595,-50.45389809963084-4.875769517473945,-50.44608153390815-4.879947558897615,-50.43923025675077-4.885570256750766,-50.43360755889762-4.892421533908147,-50.42942951747395-4.900238099630839,-50.4268566922686-4.908719567681701,-50.42598795551135-4.91754))"
202,CICATRIZ_DE_QUEIMADA,NaN,165111,2020-07-27,"(1594623600000, 1595833200000)",AWFI,CBERS-4,0.0,NaN,4.202151,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-53.38409 -12.97250, -53.38431 -12.97693, -53.38496 -12.98132, -53.38603 -12.98562, -53.38753 -12.98980, -53.38943 -12.99381, -53.39171 -12.99762, -53.39435 -13.00118, -53.39733 -13.00447, -53.40062 -13.00745, -53.40418 -13.01009, -53.40799 -13.01237, -53.41200 -13.01427, -53.41618 -13.01577, -53.42048 -13.01684, -53.42487 -13.01749, -53.42930 -13.01771, -53.43373 -13.01749, -53.43812 -13.01684, -53.44242 -13.01577, -53.44660 -13.01427, -53.45061 -13.01237, -53.45442 -13.01009, -53.45798 -13.00745, -53.46127 -13.00447, -53.46425 -13.00118, -53.46689 -12.99762, -53.46917 -12.99381, -53.47107 -12.98980, -53.47257 -12.98562, -53.47364 -12.98132, -53.47429 -12.97693, -53.47451 -12.97250, -53.47429 -12.96807, -53.47364 -12.96368, -53.47257 -12.95938, -53.47107 -12.95520, -53.46917 -12.95119, -53.46689 -12.94738, -53.46425 -12.94382, -53.46127 -12.94053, -53.45798 -12.93755, -53.45442 -12.93491, -53.45061 -12.93263, -53.44660 -12.93073, -53.44242 -12.92923, -53.43812 -12.92816, -

In [62]:
seven_month_brasil_data.drop(columns=['QUADRANT', 'PATH_ROW', 'UNIX_TIMES', 'SENSOR', 'SATELLITE', 'UC', 'geometry', 'wkt']).to_csv('10_15_7months_final.csv')